In [1]:
import os

import seaborn as sns
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import sklearn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.inspection import permutation_importance

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import fancyimpute

### Which ones will you use?

In [2]:
# Joins

# Iterative Imputation
X_sj_filled_ii = pd.read_csv("../deng_features/X_sj_filled_ii.csv")
X_iq_filled_ii = pd.read_csv("../deng_features/X_iq_filled_ii.csv")
X_train_filled_ii = pd.concat([X_sj_filled_ii, X_iq_filled_ii])

X_test_sj_filled_ii = pd.read_csv("../deng_features/X_test_sj_filled_ii.csv")
X_test_iq_filled_ii = pd.read_csv("../deng_features/X_test_iq_filled_ii.csv")
X_test_filled_ii = pd.concat([X_test_sj_filled_ii, X_test_iq_filled_ii])

# Linear Interpolation
X_sj_filled_interp = pd.read_csv("../deng_features/X_sj_filled_interp.csv")
X_iq_filled_interp = pd.read_csv("../deng_features/X_iq_filled_interp.csv")
X_train_filled_interp = pd.concat([X_sj_filled_interp, X_iq_filled_interp])

X_test_sj_filled_interp = pd.read_csv("../deng_features/X_test_sj_filled_interp.csv")
X_test_iq_filled_interp = pd.read_csv("../deng_features/X_test_iq_filled_interp.csv")
X_test_filled_interp = pd.concat([X_test_sj_filled_interp, X_test_iq_filled_interp])

# Is_missing labels csv.
sj_is_missing = pd.read_csv("../deng_features/sj_is_missing.csv")
iq_is_missing = pd.read_csv("../deng_features/iq_is_missing.csv")

X_train_is_missing = pd.read_csv("../deng_features/X_train_is_missing.csv")
X_test_is_missing = pd.read_csv("../deng_features/X_test_is_missing.csv")

In [3]:
# We won't need these columns for feature engineering.
drop_cols = ["year", "ndvi_ne", "city", "weekofyear", "month"]

X_train = pd.concat([X_sj_filled_interp, X_iq_filled_interp]).drop("Unnamed: 0", axis = 1)

# Drop year
X_train.drop(drop_cols, axis = 1, inplace = True)


####################Test #############################
X_test = X_test_filled_interp.drop("Unnamed: 0", axis = 1)

X_test.drop(drop_cols, axis = 1, inplace = True)

####################Targets###############################
y_train = pd.read_csv("../dengue_labels_train.csv")


In [4]:
X_train.head()

,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [5]:
X_test.head()

,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,-0.018900,0.102729,0.091200,78.60,298.492857,298.550000,294.527143,301.1,296.4,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,-0.012400,0.082043,0.072314,12.56,298.475714,298.557143,294.395714,300.8,296.7,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,-0.016133,0.151083,0.091529,3.66,299.455714,299.357143,295.308571,302.2,296.4,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,-0.019867,0.124329,0.125686,0.00,299.690000,299.728571,294.402857,303.0,296.9,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,0.039833,0.062267,0.075914,0.76,299.780000,299.671429,294.760000,302.3,297.3,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


### Feature Engineering Functions

In [6]:
def create_rolling(df):
    return df.rolling

### Must deal with these things:

* Having good index names for merging without errors.  Verifiability.
* Interpolation of early test rows
* Visualize these generated features to get an idea of what is going on really.
* Percent change leads to infinities when 0 is used.  How to change this value?

### First Diff

Change the backfill behavior to something ineterpolated.  Not working.

In [7]:
first_diff_train = X_train.diff().add_suffix("_d")
first_diff_train.to_csv("../deng_features/train_first_diff.csv", index = False)
first_diff_train.to_csv("../join_these_csvs/train_first_diff.csv", index = False)

first_diff_test = X_test.diff().add_suffix("_d").interpolate(limit_direction = "both", limit = 1)
first_diff_test.to_csv("../deng_features/test_first_diff.csv", index = False)
first_diff_test.to_csv("../join_these_csvs/test_first_diff.csv", index = False)

first_diff_test.head()

,ndvi_nw_d,ndvi_se_d,ndvi_sw_d,precipitation_amt_mm_d,reanalysis_air_temp_k_d,reanalysis_avg_temp_k_d,reanalysis_dew_point_temp_k_d,reanalysis_max_air_temp_k_d,reanalysis_min_air_temp_k_d,reanalysis_precip_amt_kg_per_m2_d,reanalysis_relative_humidity_percent_d,reanalysis_sat_precip_amt_mm_d,reanalysis_specific_humidity_g_per_kg_d,reanalysis_tdtr_k_d,station_avg_temp_c_d,station_diur_temp_rng_c_d,station_max_temp_c_d,station_min_temp_c_d,station_precip_mm_d
0,0.006500,-0.020686,-0.018886,-66.04,-0.017143,0.007143,-0.131429,-0.3,0.3,-3.54,-0.551429,-66.04,-0.127143,-0.557143,-0.457143,-1.500000,-3.3,0.5,-40.9
1,0.006500,-0.020686,-0.018886,-66.04,-0.017143,0.007143,-0.131429,-0.3,0.3,-3.54,-0.551429,-66.04,-0.127143,-0.557143,-0.457143,-1.500000,-3.3,0.5,-40.9
2,-0.003733,0.069040,0.019214,-8.90,0.980000,0.800000,0.912857,1.4,-0.3,-17.71,0.040000,-8.90,0.882857,1.857143,1.857143,2.228571,2.8,0.6,-31.3
3,-0.003733,-0.026755,0.034157,-3.66,0.234286,0.371429,-0.905714,0.8,0.5,-1.92,-5.254286,-3.66,-0.898571,-0.085714,0.128571,-1.514286,0.5,1.6,-2.7
4,0.059700,-0.062062,-0.049771,0.76,0.090000,-0.057143,0.357143,-0.7,0.4,2.16,1.068571,0.76,0.361429,-0.800000,-0.442857,0.814286,0.0,-1.1,83.8


### Second Diff

In [8]:
second_diff_train = X_train.diff().diff().add_suffix("_dd")
second_diff_train.to_csv("../deng_features/train_sec_diff.csv", index = False)
second_diff_train.to_csv("../join_these_csvs/train_sec_diff.csv", index = False)

second_diff_test = X_test.diff().diff().add_suffix("_dd").interpolate(limit_direction = "both", limit = 1)
second_diff_test.to_csv("../deng_features/test_sec_diff.csv", index = False)
second_diff_test.to_csv("../join_these_csvs/test_sec_diff.csv", index = False)

second_diff_test.head()

,ndvi_nw_dd,ndvi_se_dd,ndvi_sw_dd,precipitation_amt_mm_dd,reanalysis_air_temp_k_dd,reanalysis_avg_temp_k_dd,reanalysis_dew_point_temp_k_dd,reanalysis_max_air_temp_k_dd,reanalysis_min_air_temp_k_dd,reanalysis_precip_amt_kg_per_m2_dd,reanalysis_relative_humidity_percent_dd,reanalysis_sat_precip_amt_mm_dd,reanalysis_specific_humidity_g_per_kg_dd,reanalysis_tdtr_k_dd,station_avg_temp_c_dd,station_diur_temp_rng_c_dd,station_max_temp_c_dd,station_min_temp_c_dd,station_precip_mm_dd
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.023333e-02,0.089726,0.038100,57.14,0.997143,0.792857,1.044286,1.7,-0.6,-14.17,0.591429,57.14,1.010000,2.414286,2.314286,3.728571,6.1,0.1,9.6
2,-1.023333e-02,0.089726,0.038100,57.14,0.997143,0.792857,1.044286,1.7,-0.6,-14.17,0.591429,57.14,1.010000,2.414286,2.314286,3.728571,6.1,0.1,9.6
3,-1.734723e-18,-0.095795,0.014943,5.24,-0.745714,-0.428571,-1.818571,-0.6,0.8,15.79,-5.294286,5.24,-1.781429,-1.942857,-1.728571,-3.742857,-2.3,1.0,28.6
4,6.343334e-02,-0.035307,-0.083929,4.42,-0.144286,-0.428571,1.262857,-1.5,-0.1,4.08,6.322857,4.42,1.260000,-0.714286,-0.571429,2.328571,-0.5,-2.7,86.5


### pct_change

In [9]:
# Change 0's to 0.001 to prevent infinities?

zero_mapping = {0:0.01}
periods = 5

pct_change_train = X_train.replace(zero_mapping).pct_change(periods = periods).add_suffix("_pctchange")
pct_change_train.to_csv("../deng_features/train_pct_change.csv", index = False)
pct_change_train.to_csv("../join_these_csvs/train_pct_change.csv", index = False)

pct_change_test = X_test.replace(zero_mapping).pct_change(periods = periods).add_suffix("_pctchange").interpolate(limit_direction = "both", limit = 1)
pct_change_test.to_csv("../deng_features/test_pct_change.csv", index = False)
pct_change_test.to_csv("../join_these_csvs/test_pct_change.csv", index = False)

pct_change_test.head(5)

,ndvi_nw_pctchange,ndvi_se_pctchange,ndvi_sw_pctchange,precipitation_amt_mm_pctchange,reanalysis_air_temp_k_pctchange,reanalysis_avg_temp_k_pctchange,reanalysis_dew_point_temp_k_pctchange,reanalysis_max_air_temp_k_pctchange,reanalysis_min_air_temp_k_pctchange,reanalysis_precip_amt_kg_per_m2_pctchange,reanalysis_relative_humidity_percent_pctchange,reanalysis_sat_precip_amt_mm_pctchange,reanalysis_specific_humidity_g_per_kg_pctchange,reanalysis_tdtr_k_pctchange,station_avg_temp_c_pctchange,station_diur_temp_rng_c_pctchange,station_max_temp_c_pctchange,station_min_temp_c_pctchange,station_precip_mm_pctchange
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.611993,0.284939,-0.084117,-0.094529,0.004274,0.003948,0.002673,0.002657,0.004049,-0.111155,-0.028234,-0.094529,0.047025,-0.086758,0.055466,-0.267206,-0.015015,0.152074,-0.631649


## Rolling Features

What window to use for rolling features?


In [10]:




# pct change
# rolling features
# diff
# diff.diff
# 


In [11]:
# add more data by interpolating and making this 'daily' instead of 'weekly' data
# This will allow us to add more features.

In [12]:
seed_value = 42

os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)